### Alice möchte Bob 10000 satoshis senden welche Bob anschließend wieder ausgibt. Bob gibt Alice dabei verschiedene Möglichkeiten wie die Satoshis für ihn gelocked werden.

Die zweite Möglichkeit ist das Versenden der Satoshis an den Hash eines öffentlichen Schlüssels welcher durch einen von Bob kontrollierten privaten Schlüssel erzeugt wurde. Dieser Fall wird **P2PKH-Transaktion** genannt.

In [ ]:
#bitcoind -regtest -daemon -fallbackfee=0.0000025 -> startet bitcoin in regtest mode
#bcr loadwallet alice (oder bcr createwallet alice)
#bcr loadwallet bob (oder bcr createwallet bob)
#bcr -rpcwallet=alice getnweaddress "" legacy -> gibt die addresse zur Vorbereitung von welcher dann die P2PK-Locking Transaktion ausgeht
#bcr -rpcwallet=alice sendtoaddress <address> <amount in btc>
#bcr -rpcwallet=alice generatetoaddress <amount of blocks> <address>
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pkh_script
from src.ecdsa import PrivateKey
from src.crypto import hash160

**1.** In der ersten Transaktion werden von Alice die 10000 Satoshis an den Hash des von Bob kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [ ]:
#Transaktionsinputdaten
#bcr -rpcwallet=alice dumpprivkey <alice address> -> gibt den zu address gehörigen privaten Schlüssel in WIF
alice_private_key_wif_format = "cR8xne7HpYGzTphnhbvuJa6HPF3JnAKmqhHXSW4WFqaNVcTAB9e4"
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

#bcr -rpcwallet=alice listunspent -> listet alle utxo's von alice auf
#bcr -rpcwallet=alice gettransaction <tx_id> -> gibt das "Überweisungsformular" der Transaktion mit der angegebenen ID zurück.
alice_previous_tx_id_to_spent = "63eb2c763f839e6256330ed75486ee394a1f8bae619417f0e560b98060876e65"
alice_previous_tx_index_to_spent = 1
alice_previous_script_pub_key_to_spent = "76a914d200acbfd8c7776a35bd6c03bdc40c25aa5e18bc88ac"

#Transaktionsoutputdaten
alice_amount_to_spent = 10000
bob_public_key = PrivateKey(70391367530871559720919750735257129489246656383889830217409950893950252925507).get_public_key()

**1.1** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an den Hash von Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: OP_DUP OP_HASH160 <Bob's public Key> + OP_EQUALVERIFY OP_CHECKSIG generieren.

In [ ]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2pkh_script(hash160(bob_public_key.sec_format()))
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

**1.1.1** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [ ]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, alice_private_key_int_format, alice_script_sig)

#bcr -rpcwallet=alice testmempoolaccept '["tx"]' -> prüft ob die Transaktion alle Checks für die Aufnhame in den Mempool besteht
#bcr -rpcwallet=alice decoderawtransaction <tx> -> decodiert die serilaisierte Transaktion ins "Überweisungsformular"
#bcr -rpcwallet=alice sendrawtransaction <tx> -> übermittelt schließlich die Transaktion im Netzwerk so dass sie von minern für einen block aus dem mempool aufgenommen werden kann

print(f"Alice signierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

**2.** Nun möchte Bob die an den Hash seines öffentlichen Schlüssels gesendeten Satoshis weiter versenden. Dazu muss Bob im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das er berechtigt ist den Output auszugeben eine Signatur und seinen öffentlichen Schlüssel angeben. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [ ]:
#Transaktionsinputdaten
#bcr -rpcwallet=bob dumpprivkey <bob address>
bob_private_key_wif_format = "cSoDYWXxTwTbnWBpTsgWiaJbD4ZTLpJ51nHppZRFKtCK418ERJEo"
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

#bcr -rpcwallet=bob listunspent
bob_previous_tx_id_to_spent = 'a486a146b6764746d28080b824a49980e662fa1505aa0b585d7383211090931b'
bob_previous_tx_index_to_spent = 0
bob_previous_script_pub_key_to_spent = "76a91498aa473bc59b15b00c1f288e1c8d5a473533833788ac"

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [ ]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pkh_script(hash160(alice_public_key.sec_format()))
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2PKH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

**2.2** Nun kann Bob diese Transaktion mit seinem privaten Schlüssle signieren und versenden.

In [ ]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, bob_private_key_int_format, bob_script_sig)

#bcr -rpcwallet=bob testmempoolaccept '["tx"]'
#bcr -rpcwallet=bob decodetransaction <tx>
#bcr -rpcwallet=bob sendrawtransaction <tx>

print(f"Bob signierte P2PKH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")